In [2]:
import pandas
import numpy
import csv
import os
import warnings
from tqdm import tqdm

from pytest import skip

df_user_info = pandas.read_csv('user_list_more_than_20.csv')
df_anime_x = pandas.read_csv('top_and_bottom_anime.csv')
df_anime_y = pandas.read_csv('anime_matrix_true.csv')

#create main split for training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_anime_x, df_anime_y, test_size=0.2, random_state=42)

#create cross training split for building the models
X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(X_train, y_train, test_size=0.2, random_state=69)

In [3]:
#create list of anime in test set 
test_anime = []
for i in range(len(X_cross_test)):
    test_anime.append(X_cross_test.loc[i]['Id'].replace('\'',''))


In [4]:
# #X is the list of anime found in df_anime
# #y is if the the user watched the anime or not

# df_anime_x = pandas.read_csv('anime_train.csv')
# df_anime_y = pandas.read_csv('anime_matrix_true.csv')

# a = len(df_anime_x)
# b = len(df_anime_y)

# df_anime_y.drop(df_anime_y.tail(b-a).index,inplace=True)

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df_anime_x, df_anime_y, test_size=0.2, random_state=42)

In [118]:
#creates bag of words with selected features
def Create_soup_selection(x,studio=False,source=False,genre=False,va=False,all=False):
    result = x['Anime Title']

    if x['Studio'] != 'n/a' and (studio or all):
        result += ' ' + x['Studio']
    if x['Source Material'] != 'n/a' and (source or all):
        result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a' and (genre or all):
        result += ' ' + x['Genre']
    if x['VAs'] != 'n/a' and (va or all):
        result += ' ' + x['VAs']

    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    result
    return result

#sorts list of tuples by the second element
def Sort_Tuple(tup):
    # getting length of list of tuples
    lst = len(tup)
    for i in range(0, lst):
            
        for j in range(0, lst-i-1):
            if (tup[j][1] < tup[j + 1][1]):
                temp = tup[j]
                tup[j]= tup[j + 1]
                tup[j + 1]= temp
    return tup

#finds the jaccard similarity
def Jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)

In [123]:
df_anime_y = y_cross_train.copy()
users = df_anime_y.columns[1:]

print(df_anime_y.iat[1,2])

0


In [4]:
#create user list
watch_values = y_cross_train.values
users = y_cross_train.columns[1:]

#create watch list based on df_anime_y
users_watched = []
for i in tqdm(range(len(users))):
    watched = []
    for j in range(len(watch_values)):
        if(watch_values[j][i+1]):
            watched.append(str(watch_values[j][0]))
    users_watched.append(watched)

100%|██████████| 9028/9028 [00:01<00:00, 8983.64it/s]


In [ ]:
# #Creates empty anime matrix

# #create empty dataframe
# header = ['anime']
# for i in range(len(df_user_info)):
#     header.append(df_user_info.loc[i]['user'])
    
# #create row of zeros
# result = []
# for i in range(len(df_anime)):
#     row = [df_anime.loc[i]['Id'].replace('\'','')]
#     for j in range(len(df_user_info)):
#         row.append(0)
#     result.append(row)

# #output to csv
# outputdf = pandas.DataFrame(result, columns=header)
# outputdf.to_csv('anime_matrix.csv', index=False)

In [ ]:
# #fills in the true values for each anime based on user's watch list
# u = []
# for i in range(len(df_user_info)):
#     u.append(df_user_info.loc[i]['user'])
    
# for j in tqdm(range(len(u))):
#     try:
#         w = df_user_info.loc[j]["watched list"].split()
#         for i in range(len(df_anime)):
#             # print(df_matrix.at[i,'ilampan'])
#             if df_anime.loc[i]['Id'].replace('\'','') in w:
#                 df_matrix.at[i,u[j]] = 1
#     except:
#         pass

# # print(df_matrix.at[0,'ilampan'])



In [96]:
#creates anime watched true values
anime = []
for i in range(len(df_anime_x)):
    anime.append(df_anime_x.loc[i]['Id'])

users_watched = []
header = ['Id']
for i in range(len(df_user_info)):
    header.append(df_user_info.loc[i]['user'])
    try:
        users_watched.append(df_user_info.loc[i]["watched list"].split())
    except:
        users_watched.append([])    #append empty list if the user has no watch list


result = []
for i in tqdm(range(len(anime))):
    id = anime[i].replace('\'','')
    temp = [id]
    for j in range(len(users_watched)):
        if id in users_watched[j]:
            temp.append(1)
        else:
            temp.append(0)
    result.append(temp)

outputdf = pandas.DataFrame(result, columns=header)
outputdf.to_csv('anime_matrix_true.csv', index=False)

100%|██████████| 556/556 [00:17<00:00, 32.54it/s]


In [ ]:
# #read each user's watch list and creates a bag of words of all anime they watched
# result = []
# length = len(df_user_info)

# # text = 34156
# # print(text)
# # print(create_soup(df_anime.loc[df_anime['Id'] == text.index[0]]))

# for j in tqdm(range(length)):
#     try:
#         user = df_user_info.loc[j]['user']
#         watched = (df_user_info.loc[j]['watched list']).split()
#         bag = ''
#         for k in range(len(watched)):
#             if watched[k] not in test_anime:
#                 try:
#                     temp = '\'' + watched[k] + '\''
#                     index = df_anime.loc[df_anime['Id'] == temp].index[0]
#                     bag += ' ' + create_soup_reduced(df_anime.loc[index])
#                 except:
#                     pass
#     except:
#         pass
#     result.append([user,bag])

In [ ]:
# outputdf = pandas.DataFrame(result, columns=['user','bag of words'])
# outputdf.to_csv('users_bag_of_words_reduced.csv', index=False)

In [ ]:
# wa = df_user_info.loc[2]['watched list'].split()

# bag = ''
# for i in tqdm(range(len(wa))):
#     if wa[i] not in test_anime:
#         try:
#             temp = '\'' + wa[i] + '\''
#             index = df_anime.loc[df_anime['Id'] == temp].index[0]
#             bag += ' ' + create_soup(df_anime.loc[index])
#         except:
#             pass



In [ ]:
df_mini_watch_list = pandas.DataFrame(result, columns=['user', 'watch list'])

test_anime = []
for i in range(len(X_test)):
    test_anime.append(X_test.iloc[i]['Id'])

In [184]:
#read each user's watch list and creates a bag of words of all anime they watched
user_bag_result = []
length = len(users)

for j in tqdm(range(length)):
    # user = df_mini_watch_list.loc[j]['user']
    watched = users_watched[j]
    bag = ''
    for k in range(len(watched)):
        if watched[k] not in test_anime:
            try:
                temp = '\'' + str(watched[k]) + '\''
                index = df_anime_x.loc[df_anime_x['Id'] == temp].index[0]
                bag += ' ' + Create_soup_selection(df_anime_x.loc[index],all=True)
            except:
                pass
    user_bag_result.append([users[j],bag])

#creates a bag of words for each anime
anime_bag_result = []
for i in tqdm(range(len(df_anime_x))):
    anime = df_anime_x.iloc[i]['Id'].replace('\'','')
    bag = Create_soup_selection(df_anime_x.iloc[i],all=True)
    anime_bag_result.append([anime,bag])

#output the user bag of words to csv
outputdf = pandas.DataFrame(user_bag_result, columns=['user','bag of words'])
outputdf.to_csv('users_bag_of_words_all.csv', index=False)

#output the anime bag of words to csv
outputdf = pandas.DataFrame(anime_bag_result, columns=['Id','bag of words'])
outputdf.to_csv('anime_bag_of_words_all.csv', index=False)

100%|██████████| 556/556 [00:00<00:00, 7935.59it/s]


In [185]:

rows =[]
for i in range(len(X_cross_test)):
    rows.append(X_cross_test.iloc[i]['Id'].replace('\'',''))

In [187]:
#compute all distances and dump to file
test = X_cross_test.copy()
train = X_cross_train.copy()

all_distances = []
for j in tqdm(range(len(test))):
    bag1 = Create_soup_selection(test.iloc[j],all=True).split()
    distances = []
    for i in range(len(train)):
        bag2 = Create_soup_selection(train.iloc[i],all=True).split()
        distances.append((train.iloc[i]['Id'].replace('\'',''),Jaccard(list(bag1),list(bag2))))
    Sort_Tuple(distances)
    all_distances.append(distances)

import pickle

# open a file, where you ant to store the data
file = open('knn_distances_all', 'wb')

# dump information to that file
pickle.dump(all_distances, file)

# close the file
file.close()

100%|██████████| 89/89 [00:04<00:00, 21.14it/s]


In [188]:
import pickle

# open a file, where you stored the pickled data
file = open('knn_distances_all', 'rb')

# dump information to that file
all_distances = pickle.load(file)

# close the file
file.close()

In [202]:
#KNN implementation

#make empty prediction matrix
#add anime Id column
#for every user
#   for each list of distances
#       find value of sum and divide by k
#       if sum / k is >= 0.5 predict 1, else predict 0
#   append to list of predictions

# print(type(distances[1][0]))
# temp = df_anime_y.loc[df_anime_y['Id'] == 32937]['ilampan']
# print(df_anime_y.loc[df_anime_y['Id'] == 32937])
for knn in tqdm(range(20,26)):
    neighbors = knn
    prediction_result = []
    for j in (range(len(users))):
        col = users[j]
        watched = users_watched[j]
        predictions = []
        for k in range(len(all_distances)):
            distances = all_distances[k]
            #compute the sum
            neigh = []
            for i in range(neighbors):
                neigh.append(distances[i][0])
            value = len(list(set(neigh).intersection(watched)))
            if (value / neighbors) >= 0.5:
                predictions.append(1)
            else:
                predictions.append(0)
        prediction_result.append(predictions)

    #create df_predictions
    warnings.simplefilter(action='ignore', category=pandas.errors.PerformanceWarning)
    df_predictions = pandas.DataFrame()
    df_predictions['Id'] = rows
    for i in range(len(users)):
        col = users[i]
        df_predictions[col] = prediction_result[i]

    #write to csv
    outputdf = df_predictions.copy()
    outputdf.to_csv(f'predictions_matrix_{neighbors}nn.csv', index=False)

100%|██████████| 6/6 [00:44<00:00,  7.48s/it]


In [ ]:
df_test = pandas.DataFrame()
df_test['Id'] = rows
print(len(prediction_result))
df_temp = pandas.DataFrame(prediction_result,columns=users)
# df_teee = pandas.DataFrame(prediction_result, index=False, columns=users)
pandas.concat([df_test, pandas.DataFrame(prediction_result, index=df_test.index, columns=rows)], axis=1)

In [ ]:
# temp = all_distances[0]
# watched = users_watched[0]
# print(watched)
# t = []
# t.append(str(30))
# print(t[0] in watched)
# # print(watched)
# print(temp[0:neighbors])
# neigh = []
# for i in range(neighbors):
#     neigh.append(temp[i][0])
# print('neighbors = ',neigh)
# print(t[0] in neigh)
# print('intersect = ',len(list(set(watched).intersection(t))))

In [ ]:
df_predictions = pandas.DataFrame()
df_predictions['Id'] = rows
print(df_predictions.head)

In [9]:
df_true = pandas.read_csv('anime_matrix_true.csv')
df_test = df_true.copy()



In [9]:

#caluclate accuracy
value = 0
prediction_matrix = pandas.read_csv('predictions_matrix_adaboost.csv')
pred = prediction_matrix.values
true = y_test.values
length_row = len(pred)
length_col = len(pred[0])
for i in tqdm(range(length_row)):
    for j in range(1,length_col):
        if pred[i,j] == true[i,j]:
            value += 1

print(value)
print(value / ((length_row) * (length_col - 1)))
# print('zeros = ',zeros)
# print('ones = ',ones)
# print('true ones = ',true_ones)
print((length_row) * (length_col - 1))

100%|██████████| 89/89 [00:00<00:00, 458.35it/s]

507902
0.6321883288959588
803403


In [12]:
df_true = pandas.read_csv('anime_matrix_true.csv')

true_values = df_true.values

In [210]:

watched_count = []
print(true_values[0][0])
for i in tqdm(range(len(true_values))):
    value = sum(true_values[i][1:])
    watched_count.append((true_values[i][0],value))


32937


100%|██████████| 4375/4375 [00:01<00:00, 3070.34it/s]


In [275]:
Sort_Tuple(watched_count)
# for each in watched_count:
#     print(each)

top_anime = []
watched_count_slice = watched_count[0:278]
for each in watched_count_slice:
    top_anime.append(str(each[0]))


# print(top_anime)

In [16]:
df_temp = pandas.read_csv('user_list_more_than_20.csv')

print(df_temp['watched count'].median())

print(df_temp['watched count'].quantile([0.25,0.5,0.75]))

278.0
0.25    140.0
0.50    278.0
0.75    502.0
Name: watched count, dtype: float64


In [88]:
df_anime_temp = pandas.read_csv('new_anime.csv')
anime_values = df_anime_temp.values
print(len(anime_values))



result = []
indicies = []
new_array = numpy.copy(anime_values)
for i in tqdm(range(len(anime_values))):
    if anime_values[i][0].replace('\'','') in top_anime:
        result.append(anime_values[i])
        indicies.append(i)

4375


100%|██████████| 4375/4375 [00:00<00:00, 364211.74it/s]


In [92]:
# for i in tqdm(range(len(df_anime_temp))):
#     if df_anime_temp.loc[i]['Id'].replace()
bottom_anime = df_anime_temp.drop(indicies).values
bottom_anime = pandas.DataFrame(bottom_anime, columns=df_anime_temp.columns)
bottom_sample = bottom_anime.sample(278, random_state=69)

top_sample = pandas.DataFrame(result, columns=df_anime_temp.columns)

frames = [top_sample, bottom_sample]

top_and_bottom = pandas.concat(frames)

In [94]:
print(top_and_bottom.head)

outputdf = top_and_bottom.copy()
outputdf.to_csv('top_and_bottom_anime.csv', index=False)

<bound method NDFrame.head of            Id                           Anime Title Rating  \
0     '32937'  'KonoSubarashiiSekainiShukufukuwo!2'   8.27   
1     '33206'        'Kobayashi-sanChinoMaidDragon'   7.99   
2     '32615'                          'YoujoSenki'   7.97   
3     '33487'               'Masamune-kunnoRevenge'   6.74   
4     '32949'                        'KuzunoHonkai'   7.17   
...       ...                                   ...    ...   
509   '38544'                         'EgaonoDaika'   6.04   
1435   '8890'        'Nippon-ichinoOtokonoTamashii'   5.98   
3109  '13203'    'LupintheThird:MineFujikotoIuOnna'   7.77   
3249  '16211'              'BakumatsuGijindenRoman'   6.13   
3165  '13115'                 'Hakuouki:Reimeiroku'   7.44   

                   Studio Source Material Demographic  \
0              StudioDeen      Lightnovel       'n/a'   
1          KyotoAnimation           Manga       'n/a'   
2                     Nut      Lightnovel       'n/a' 

In [280]:
def Remove_Punc(word):
    punc = '[],\'\"'
    
    result = word
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")
    return result


In [290]:
#playing with ada
genreDict = dict()
studioDict = dict() 
vaDict = dict() 
sourceDict = dict()
scoreDict = dict()

for i in tqdm(range(len(df_anime_x))):
    # print(df_anime_x.loc[i])
    # print(watched_count[i][1])
    # print(df_anime_x.loc[i]['Genre'])
    genreWords = Remove_Punc(df_anime_x.loc[i]['Genre']).split()
    studioWords = Remove_Punc(df_anime_x.loc[i]['Studio']).split()
    vaWords = Remove_Punc(df_anime_x.loc[i]['VAs']).split()
    sourceWords = Remove_Punc(df_anime_x.loc[i]['Source Material']).split()
    for each in genreWords:
        if each in genreDict:
            genreDict[each] = genreDict.get(each) + watched_count[i][1]
        else:
            genreDict[each] = watched_count[i][1]
    for each in studioWords:
        if each in studioDict:
            studioDict[each] = studioDict.get(each) + watched_count[i][1]
        else:
            studioDict[each] = watched_count[i][1]
    for each in vaWords:
        if each in vaDict:
            vaDict[each] = vaDict.get(each) + watched_count[i][1]
        else:
            vaDict[each] = watched_count[i][1]
    for each in sourceWords:
        if each in vaDict:
            sourceDict[each] = sourceDict.get(each) + watched_count[i][1]
        else:
            sourceDict[each] = watched_count[i][1]



100%|██████████| 556/556 [00:00<00:00, 3703.30it/s]

7296
['Adventure', 'Comedy', 'Fantasy', 'Isekai', 'Parody', 'Reincarnation']
7063
['Fantasy', 'SliceofLife']
6931
['Action', 'Isekai', 'Military', 'Reincarnation']
6609
['Comedy', 'Romance', 'Harem', 'School', 'Shounen']
6482
['Drama', 'Romance', 'School', 'Seinen']
6363
['Action', 'Drama', 'Gore', 'Military', 'Survival', 'Shounen']
6323
['Action', 'School', 'SuperPower', 'Shounen']
6150
['Comedy', 'Drama', 'Romance', 'Ecchi']
6117
['Action', 'Fantasy', 'School']
6030
['Romance', 'School']
6015
['Drama', 'Mystery', 'HighStakesGame', 'Psychological', 'School', 'StrategyGame', 'Shounen']
5968
['Adventure', 'Drama', 'Fantasy', 'Mystery', 'Sci-Fi']
5913
['Drama', 'Psychological', 'School']
5846
['Comedy', 'Romance', 'School', 'Shounen']
5838
['Comedy', 'Romance', 'OtakuCulture', 'School', 'VideoGame']
5834
['Action', 'Comedy', 'Fantasy', 'Shounen']
5773
['Gourmet', 'Ecchi', 'School', 'Shounen']
5708
['Fantasy', 'Shounen']
5687
['Comedy', 'Workplace']
5594
['Action', 'Drama', 'Sci-Fi', 'Gor

In [296]:
sorted_genres = sorted(genreDict.items(), key=lambda x:x[1])
print(sorted_genres)
# print('genre length: ',len(sorted_genres))

# for each in sorted_genres[(70//2):]:
    # print(each[])

sorted_studio = sorted(studioDict.items(), key=lambda x:x[1])
print(sorted_studio)
print('studio length: ',len(sorted_studio))

sorted_vas = sorted(vaDict.items(), key=lambda x:x[1])
print(sorted_vas)
print('vas length: ',len(sorted_vas))

sorted_source = sorted(sourceDict.items(), key=lambda x:x[1])
print(sorted_source)
print('source length: ',len(sorted_source))

[('Josei', 1309), ('(Male)', 1940), ('BoysLove', 3609), ('Pets', 4584), ('GirlsLove', 6333), ('Medical', 6691), ('AvantGarde', 8201), ('PerformingArts', 9258), ('Childcare', 9336), ('ReverseHarem', 9535), ('(Female)', 9785), ('Educational', 10415), ('Racing', 11098), ('Idols', 11725), ('VisualArts', 11759), ('Detective', 12252), ('Delinquents', 14063), ('MahouShoujo', 17015), ('OrganizedCrime', 18668), ('MartialArts', 19486), ('Space', 19763), ('Samurai', 20217), ('Anthropomorphic', 20431), ('OtakuCulture', 21448), ('CGDCT', 23018), ('GagHumor', 24309), ('Vampire', 24619), ('TeamSports', 25229), ('HighStakesGame', 25967), ('Iyashikei', 26369), ('Workplace', 26561), ('Gourmet', 27857), ('StrategyGame', 32731), ('RomanticSubtext', 33819), ('Parody', 36139), ('Sports', 40457), ('Music', 40705), ('LovePolygon', 41388), ('AdultCast', 43300), ('VideoGame', 48660), ('TimeTravel', 51248), ('Reincarnation', 51515), ('Shoujo', 51632), ('Horror', 57102), ('Mythology', 58658), ('Survival', 60599),